In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "root"
password = "KPdFsW8nv8"
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32355, 'AAC', 'animals')

# Default data retrieval
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare Dashboard')

# Layout
app.layout = html.Div([
    html.img(src='Grazioso_Salvare_Logo.png', style={'width': '25%', 'height': '25%'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div("Unique Identifier: Nathaniel Gratton - CS-340 MongoDB Dashboard"),
    html.Hr(),
    
    # Row for Filter and Data Table
    html.Div([
        # Filters on the left side
        html.Div([
            html.Label('Interactive Filter Options'),
            dcc.RadioItems(
                id='filter-options',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                    {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                value='reset',
                labelStyle={'display': 'block'}  # Vertically stack radio buttons
            )
        ], className="three columns", style={'width': '20%', 'display': 'inline-block', 'padding': '20px'}),

        # DataTable on the right side
        html.Div([
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                page_size=10,  # Show 10 rows per page
                sort_action="native",  # Enable sorting
                filter_action="native",  # Enable filtering
                row_selectable="single",  # Allow selecting a single row
                selected_rows=[0],  # Default select the first row
                style_table={'overflowX': 'auto'},  # Make the table scrollable horizontally
                style_cell={'textAlign': 'left'},
            ),
        ], className="nine columns", style={'width': '75%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'width': '100%'}),  # Use flexbox for horizontal alignment
    
    html.Br(),
    html.Hr(),

    # Row for Pie Chart and Map
    html.Div([
        # Pie chart on the left
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className="six columns", style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),

        # Geolocation map on the right
        html.Div([
            html.Div(id='map-id')
        ], className="six columns", style={'width': '50%', 'display': 'inline-block', 'padding': '10px'})
    ], style={'display': 'flex', 'width': '100%'}),
    
    html.Br(),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter Callback to Update Data Table, Pie Chart, and Map
@app.callback(
    [Output('datatable-id', 'data'),
     Output('pie-chart', 'figure'),
     Output('map-id', 'children')],
    [Input('filter-options', 'value'),
     Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_dashboard(filter_value, viewData, index):
    # Apply filters based on the selected rescue type
    query = {}
    if filter_value == 'water':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
    elif filter_value == 'mountain':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_value == 'disaster':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    
    # Fetch filtered data
    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)
    
    # Group breeds with less than 1% of total into "Other"
    breed_counts = filtered_df['breed'].value_counts()
    total_count = breed_counts.sum()
    threshold = total_count * 0.01
    breed_counts_filtered = breed_counts[breed_counts > threshold]
    breed_counts_filtered['Other'] = breed_counts[breed_counts <= threshold].sum()

    # Update Pie Chart with total count in tooltips
    pie_chart = px.pie(
        names=breed_counts_filtered.index, 
        values=breed_counts_filtered.values, 
        title='Breed Distribution',
        labels={'value': 'Count'}
    )
    pie_chart.update_traces(textinfo='label+percent', hovertemplate='%{label}: %{value} animals')

    # Map Update
    if viewData is None or len(viewData) == 0:
        return filtered_df.to_dict('records'), pie_chart, [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Popup([html.H1("No Data Available")])
        ])]

    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    if row >= len(dff):
        row = 0

    lat, lon = dff.iloc[row].get('location_lat'), dff.iloc[row].get('location_long')
    if pd.isnull(lat) or pd.isnull(lon) or not isinstance(lat, (int, float)) or not isinstance(lon, (int, float)):
        return filtered_df.to_dict('records'), pie_chart, [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Popup([html.H1("Invalid Coordinates")])
        ])]
    
    map_view = dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(f"{dff.iloc[row]['name']} - {dff.iloc[row]['breed']}"),  # Tooltip includes name and breed
            dl.Popup([html.H1(f"Animal Name: {dff.iloc[row]['name']}"), html.P(f"Breed: {dff.iloc[row]['breed']}")])
        ])
    ])

    return filtered_df.to_dict('records'), pie_chart, map_view

# Run the Dash app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:30801/
